In [115]:
# pip install pymongo

In [116]:
import requests
from pymongo import MongoClient
import json

In [117]:
client = MongoClient('mongodb://localhost:27017/')
database = client['IndeedJobListing']
collection = database['Hello']

In [111]:
job_search_keyword = ['Python+developer']
api_url = "https://api.scrapingdog.com/indeed"
api_key = "659b01009794a261d9eaee40"
pagination_url = "https://in.indeed.com/jobs?q={}&l=&from=searchOnHP&vjk=71d709239fecfc67"
job_search_url = pagination_url.format(job_search_keyword[0], 0)

# Set up the parameters
params = {"api_key": api_key, "url": job_search_url}
print(params)
# Make the HTTP GET request
response = requests.get(api_url, params=params)



{'api_key': '659b01009794a261d9eaee40', 'url': 'https://in.indeed.com/jobs?q=Python+developer&l=&from=searchOnHP&vjk=71d709239fecfc67'}


In [112]:
# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON content
    json_response = response.json()
    print(json_response)
else:
    print(f"Error: {response.status_code}")
    print(response.text)


[{'jobTitle': 'Backend Python Developer - WFH', 'jobLink': 'https://www.indeed.com/rc/clk?jk=5a61fdba828e98d8&bb=KcK9d95sIneKt9Eh860eYHa0xX8wVZ6AXi-eAIGSzGaFL-jasnDR76xqHzHYjxtU&xkcb=SoDY67M3G0kXi52YGh0LbzkdCdPP&fccid=0e2f08d304beb5b1&cmp=Uplers&ti=Back+End+Developer&vjs=3', 'companyName': 'Uplers', 'companyLocation': 'Remote', 'jobDescription': ' \n Salary:* USD 2,470/ month (Based on experience).\n Shift: US Full Shift (08:00 PM to 05:00 AM IST).\n Contract Duration: Full-time, 12 months.\n', 'Salary': '', 'jobMetaData': [], 'jobPosting': 'Hiring ongoing'}, {'jobTitle': 'Python Developer', 'jobLink': 'https://www.indeed.com/rc/clk?jk=922a01bc2221f840&bb=TrWZzlcqs1M13wAg-hMdY4pWVkVtyjLJZDiWDAPsLVL0pgRtYsyrJdYROekY1n8O&xkcb=SoB_67M3G0kXi4WYGh0JbzkdCdPP&fccid=146443e77d8c0778&vjs=3', 'companyName': 'Virtusa', 'companyLocation': 'Hybrid remote in Bengaluru, Karnataka', 'jobDescription': ' \n Python Developer responsibilities include writing and testing code, debugging programs and integr

In [98]:
# # Install if you have never used these: unblock the lines below to install if needed
# !pip install webdriver-manager
# !pip3 install lxml
# !pip3 install selenium
# !pip3 install webdriver_manager
# !pip install --upgrade pip
# !pip install -U selenium
# !pip install msedge-selenium-tools


In [99]:
# # --------- import necessary modules -------

# # For webscraping
# from bs4 import BeautifulSoup

# # Parsing and creating xml data
# from lxml import etree as et

# # Store data as a csv file written out
# from csv import writer

# # In general to use with timing our function calls to Indeed
# import time

# # Assist with creating incremental timing for our scraping to seem more human
# from time import sleep

# # Dataframe stuff
# import pandas as pd

# # Random integer for more realistic timing for clicks, buttons and searches during scraping
# from random import randint

# # Multi Threading
# import threading

# # Threading:
# from concurrent.futures import ThreadPoolExecutor, wait

In [100]:
# import selenium

# # Check version I am running
# selenium.__version__

In [101]:
# # Selenium 4:

# from selenium import webdriver

# # Starting/Stopping Driver: can specify ports or location but not remote access
# from selenium.webdriver.chrome.service import Service as ChromeService

# # Manages Binaries needed for WebDriver without installing anything directly
# from webdriver_manager.chrome import ChromeDriverManager

In [102]:
# # Allows searchs similar to beautiful soup: find_all
# from selenium.webdriver.common.by import By

# # Try to establish wait times for the page to load
# from selenium.webdriver.support.ui import WebDriverWait

# # Wait for specific condition based on defined task: web elements, boolean are examples
# from selenium.webdriver.support import expected_conditions as EC

# # Used for keyboard movements, up/down, left/right,delete, etc
# from selenium.webdriver.common.keys import Keys

# # Locate elements on page and throw error if they do not exist
# from selenium.common.exceptions import NoSuchElementException

In [103]:
# # Allows you to cusotmize: ingonito mode, maximize window size, headless browser, disable certain features, etc
# option= webdriver.ChromeOptions()

# # Going undercover:
# option.add_argument("--incognito")


# # # Consider this if the application works and you know how it works for speed ups and rendering!

# # option.add_argument('--headless=chrome')

In [104]:
    # # Define job search keyword for Data Analyst
    # job_search_keyword = ['Python+developer']

    # # Finding position, radius=35 miles, sort by date and starting page
    # # pagination_url = 'https://www.indeed.com/jobs?q={}&l=' 
    # pagination_url = 'https://in.indeed.com/jobs?q={}&l=&from=searchOnHP&vjk=71d709239fecfc67'  

    # # Print the pagination URL for Data Analyst
    # print(pagination_url.format(job_search_keyword[0], 0))


In [105]:
# start = time.time()

# job = 'Python+developer'

# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=option)

# driver.get(pagination_url.format(job_search_keyword[0], 0))

# # Add a sleep to simulate human-like interactions
# sleep(randint(2, 6))

# # Find the job count element
# job_count_element = driver.find_element(By.CLASS_NAME, 'jobsearch-JobCountAndSortPane-jobCount')

# # Extract the job count text
# job_count_text = job_count_element.text

# # Extract the numerical job count
# # job_count = int(job_count_text.split(' ')[0])
# # Extract the numerical job count, removing commas
# job_count = int(job_count_text.split(' ')[0].replace(',', ''))



# # Calculate the maximum iterable pages
# max_iter_pgs = job_count // 15

# # Continue with the rest of your code if needed

# driver.quit()  # Closing the browser session

# end = time.time()

# print(end - start, 'seconds to complete action!')
# print('-----------------------')
# print('Max Iterable Pages for this search:', max_iter_pgs)
 


In [106]:
# print(job_count)

In [107]:
# def try_except(func, default_value):
#     try:
#         return func()
#     except NoSuchElementException:
#         return default_value

# start = time.time()

# job = 'Python+developer'

# job_lst = []
# salary_list = []

# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=option)
# sleep(randint(2, 6))

# # Set a limit for the number of pages to print debug messages
# debug_pages_limit = 13
# pages_processed = 0

# for i in range(0, max_iter_pgs):
#     driver.get(pagination_url.format(job, '', i * 10))

#     print(f"Page {i+1} loaded successfully.")

#     sleep(randint(2, 4))

#     job_page = driver.find_element(By.ID, "mosaic-jobResults")
#     jobs = job_page.find_elements(By.CLASS_NAME, "job_seen_beacon")

#     for jj in jobs:
#         job_title = jj.find_element(By.CLASS_NAME, "jobTitle")

#         print(f"Processing job: {job_title.text}")

#         job_lst.append([
#             job_title.text,
#             job_title.find_element(By.CSS_SELECTOR, "a").get_attribute("href"),
#             job_title.find_element(By.CSS_SELECTOR, "a").get_attribute("id"),
#             try_except(lambda: jj.find_element(By.CLASS_NAME, "companyName").text, None),
#             job_title.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
#         ])

#         try:
#             salary_list.append(jj.find_element(By.CLASS_NAME, "salary-snippet-container").text)
#         except NoSuchElementException:
#             try:
#                 salary_list.append(jj.find_element(By.CLASS_NAME, "estimated-salary").text)
#             except NoSuchElementException:
#                 salary_list.append(None)

#     # Increment the pages processed counter
#     pages_processed += 1

#     # Check if we have reached the debug pages limit
#     if pages_processed >= debug_pages_limit:
#         print(f"Debug messages truncated after processing {debug_pages_limit} pages.")
#         break

# driver.quit()

# end = time.time()

# print(end - start, 'seconds to complete Query!')


In [108]:
# print(job_lst[0:4])
# print(salary_list[0:4])